In [ ]:
import pandas as pd
import pywhatkit as wpp
from datetime import datetime, timedelta

#SetOptions para ver los dataframes
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth',False)

In [ ]:
def dias_alertas():
    hoy = datetime.now()
    dias_semana = {'lunes': 0, 'martes': 1, 'miércoles': 2, 'jueves': 3, 'viernes': 4, 'sábado': 5, 'domingo': 6}

    # Encuentra el último martes y jueves
    last_tuesday = hoy - timedelta(days=(hoy.weekday() - dias_semana['martes']) % 7)
    last_thursday = hoy - timedelta(days=(hoy.weekday() - dias_semana['jueves']) % 7)

    # Devuelve la fecha más reciente entre el último martes y jueves
    return max(last_tuesday, last_thursday).strftime('%d')

In [ ]:
def clean_alertas(df):
    # Adios a los registros. Filtro solo Cadereyta
    df = df.drop(columns=['Registro 1', 'Registro 2', 'Registro 3', 'Registro 4', 'Registro 5', 'Subproyecto'])
    df = df[df['Centro Enlace'].isin(['CADEREYTA', 'CE CADEREYTA', 'CC CADEREYTA'])]
    
    # Juntar las materias
    # inactivos_agrup = df.groupby(["Matricula"])["Grupo"].agg(lambda x: ", ".join(x)).reset_index().rename(columns={'Grupo':'Materias'})
    df['Materias'] = df.groupby(["Matricula", 'Estatus'])["Grupo"].transform(lambda x: ', '.join(x)) #.agg(lambda x: ", ".join(x)).reset_index().rename(columns={'Grupo':'Materias'})
    
    # df.shape #Materias Agrupadas en la última colunmna
    
    # Reacomodo de las materias
    df["Grupo"]  = df["Materias"]
    df.drop(columns = ["Materias", 'Modelo'], inplace=True)
    df = df.drop_duplicates(subset= ['Nombre', 'Estatus'])
    df.shape #sin duplicados 
    return df

In [ ]:
def clean_database(db):
    #Renombrar el nombre de la columna NOMBRE
    db.rename(columns = {'NOMBRE                                                                                                                                      ' : "NOMBRE", 'MATRICULA' : "Matricula"}, inplace= True)

    # Columnas importantes
    db = db[['NOMBRE', 'Matricula', 'CE CANALIZADO', 'CARRERA', 'M4', 'Correo', 'Teléfono', 'Teléfono2', 'Teléfono3', 'CURP', 'Agente ', 'Asesor Académico', 'Asesor Financiero ', 'Asesor Escolar']]

    # Filtro Cadereyta
    # db = db[db['CE CANALIZADO'] == 'CE CADEREYTA']
    cade = ('CADEREYTA', 'CE CADEREYTA', 'CC CADEREYTA')
    db = db[db['CE CANALIZADO'].str.contains('|'.join(cade), case = False, na = False)]
    return db

In [ ]:
def filtro_estatus(df):
    option = int(input('1.- Inactivos \n2.- Ausentes \n3.- Regular \n 9.- Out'))

    if option == 1:
        # Inactivos
        option = ('Inactivo',)
    elif option == 2:
        # Ausente - Posible Reprobado
        option = ('Ausente', 'Posible Reprobado')
    elif option == 3:
        # Regular
        option = ('Regular',)
    return df[df['Estatus'].isin(option)]

In [ ]:
path = 'Data/'

try:
    database = pd.read_csv(path + "BASE CNCI.csv") 
except FileNotFoundError or FileExistsError: 
    %run DatabaseCNCI.ipynb
    print('Database CNCI.csv Created')
    database = pd.read_csv(path + "BASE CNCI.csv")
    
try: 
    alertas = pd.read_csv(path + "Alertas.csv", index_col= 0)
except FileNotFoundError or FileExistsError:
    alertas = pd.read_excel(path + 'Main/Alertas tempranas 30 de Abril 2024.xlsx', sheet_name='General')
    clean_alertas(alertas).to_csv(path + 'Alertas.csv')
    print('Alertas csv Created')
    alertas = pd.read_csv(path + "Alertas.csv", index_col= 0)


In [ ]:
imagen = 'img/Cerro.jpeg'
mencion_stat = ''
path2 = 'infodata/'

#Inicializar la infodata
try:
    matric_error = pd.read_csv(path2 + 'matriculas_error.csv')
    matric_yacontactadas = pd.read_csv(path2+ 'matriculas_yacontactadas.csv') 
    
    matric_error = matric_error.dropna().values.flatten().tolist()
    matric_yacontactadas = matric_yacontactadas.dropna().values.flatten().tolist() 
except FileNotFoundError:
    matric_error = pd.DataFrame()
    matric_yacontactadas = pd.DataFrame()

In [ ]:
database = clean_database(database)
alertas = filtro_estatus(alertas)

In [ ]:
# Merge nuestros df, con el fin de no llamar a dos df en el for.
alertas_merged = pd.merge(alertas, database[['Matricula','Teléfono', 'Teléfono2']] , on = 'Matricula')

Ciclo de mensajes

In [ ]:
for index, row in alertas_merged.iterrows():
    alumno = row['Nombre']
    matricula = row['Matricula']
    materias = row['Grupo']
    telefono = row['Teléfono']
    status = row['Estatus']
    print(telefono)
    if not pd.isna(telefono):
        telefono = '+52 ' + str(telefono)
    if matricula in matric_error:
        if not pd.isna(row['Teléfono2']):
            telefono2 = '+52 ' + str(row['Teléfono2'])
            telefono = telefono2
            print('Matricula con error: {}, Alumno: {}'.format(matricula, alumno))
            
    if status == ('Inactivo'):
        mencion_stat = 'veo que aún no has ingresado a tus materias: *' + materias +'*'
    elif status == ('Ausente'):
        mencion_stat = 'veo que ya has ingresado a tus materias: *'+ materias +'* \n ¿Has logrado avanzar con tus tareas?'
    elif status == ('Posible Reprobado'):
        mencion_stat == '¿Cómo vas con tus tareas?'
        
    if matricula in matric_yacontactadas:
        print('Matricula antes contactada: {}, Alumno: {} '.format(matricula, alumno))    
    else:
        mensaje = 'Hola, que tal ' + alumno + ' ' + matricula + ', soy _Román Rodríguez_ Tutor Presencial del CNCI, me comunico contigo ya que ' + mencion_stat + ' ¿Hay algo en lo que pueda apoyarte?'
        print('Enviado a: ', matricula, alumno, '\n')
        #wpp.sendwhats_image(telefono, imagen, mensaje, 25, True)
        #print(telefono, matricula, alumno)